<a href="https://colab.research.google.com/github/HelloWorld33333/Pork/blob/main/AI_07_%EB%B0%95%ED%83%9C%EA%B7%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT finetuning

In [ ]:
#!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
#!wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
data_set1 = '/content/drive/MyDrive/Colab Notebooks/P4/감성대화말뭉치(원시데이터)_Training.xlsx'

df = pd.read_excel(data_set1, usecols = [6, 8, 10, 12])

In [ ]:
data_set2 = '/content/drive/MyDrive/Colab Notebooks/P4/감성대화말뭉치(최종데이터)_Training.xlsx'

df_2 = pd.read_excel(data_set2, usecols = [5, 7, 9, 11, 13])

In [ ]:
df = df.astype(str)
df_2 = df_2.astype(str)

In [ ]:
df['사람문장3'] = df['사람문장3'].replace('nan', '')

In [ ]:
df['sentence'] = df[['사람문장1', '사람문장2', '사람문장3']].apply(' '.join, axis=1)

df_clone_1 = df[['감정_대분류', 'sentence']]
df_clone_1

,감정_대분류,sentence
0,기쁨,지금 난 기분이 너무 좋아. 오늘부터 연휴가 시작됐거든. 내일 회사 안 간다. 그냥...
1,기쁨,나도 조카가 생겨! 너무 기뻐. 응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 ...
2,불안,거래처와의 다음 계약이 무산될까봐 불안해. 딱히 이유는 없어. 그냥 나 혼자 불안한...
3,슬픔,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...
4,슬픔,노년에 재정적으로 이루어 놓은게 없어서 걱정이야. 정말이지 막막해.
...,...,...
74851,분노,부모님의 건강을 못 지킨 나에게 화가 나. 있는 돈으로 병원비를 지원을 할래.
74852,상처,요즘 취업 준비하면서 잘사는 집 사람들이 부럽더라. 낙하산으로 들어가고 하니까. 우...
74853,분노,노인 대상 보험 가입이 거절돼서 화가 나. 나는 나이만 많을 뿐이고 별다른 질병이 ...
74854,분노,은퇴가 코앞인데 돈 들어갈 곳은 천지네. 집사람은 경력단절 된 지 오래이고 애들은...


In [ ]:
df_2['사람문장3'] = df_2['사람문장3'].replace('nan', '')
df_2['사람문장4'] = df_2['사람문장4'].replace('nan', '')

In [ ]:
df_2['sentence'] = df_2[['사람문장1', '사람문장2', '사람문장3', '사람문장4']].apply(' '.join, axis=1)

df_clone_2 = df_2[['감정_대분류', 'sentence']]
df_clone_2['감정_대분류'].value_counts()

불안     7328
분노     6920
슬픔     6915
기쁨     6730
상처     6624
당황     6360
불안        1
기쁨        1
Name: 감정_대분류, dtype: int64

In [ ]:
df_clone_2.groupby(['감정_대분류']).count()

class_group = df_clone_2.groupby(['감정_대분류'])
class_group.groups

{'기쁨': [0, 3, 4, 14, 23, 35, 42, 56, 61, 63, 65, 66, 74, 77, 81, 90, 92, 93, 103, 109, 146, 147, 159, 163, 165, 179, 180, 198, 199, 205, 206, 207, 210, 220, 227, 229, 248, 250, 257, 261, 263, 267, 271, 283, 295, 301, 302, 304, 309, 313, 330, 335, 340, 364, 367, 377, 378, 388, 391, 400, 403, 414, 419, 425, 426, 427, 429, 431, 438, 440, 443, 444, 452, 464, 474, 481, 502, 513, 519, 522, 526, 540, 542, 543, 545, 550, 558, 563, 564, 567, 576, 577, 586, 604, 605, 611, 612, 624, 625, 630, ...], '기쁨 ': [35511], '당황': [2, 11, 12, 22, 24, 29, 50, 52, 64, 67, 70, 78, 86, 95, 101, 108, 111, 116, 126, 129, 131, 132, 136, 141, 143, 145, 149, 151, 154, 164, 169, 170, 172, 173, 183, 192, 194, 200, 203, 208, 209, 213, 219, 226, 233, 235, 241, 253, 255, 268, 269, 276, 280, 294, 299, 307, 308, 311, 315, 316, 317, 325, 327, 331, 336, 343, 344, 347, 357, 359, 382, 387, 390, 393, 406, 408, 421, 434, 437, 458, 459, 460, 466, 470, 473, 478, 482, 486, 488, 491, 495, 500, 505, 507, 508, 516, 533, 536, 538, 549,

In [ ]:
df_clone_3 = df_clone_2.drop([df_clone_2.index[35511], df_clone_2.index[34527]])
df_clone_3['감정_대분류'].value_counts()

불안    7328
분노    6920
슬픔    6915
기쁨    6730
상처    6624
당황    6360
Name: 감정_대분류, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoder.fit(df_clone_1['감정_대분류'])
labels = encoder.transform(df_clone_1['감정_대분류'])

print(encoder.inverse_transform([0,1,2,3,4,5]))

df_clone_1['감정_대분류'] = labels

print(df_clone_1['감정_대분류'].value_counts())


encoder = LabelEncoder()

encoder.fit(df_clone_3['감정_대분류'])
labels = encoder.transform(df_clone_3['감정_대분류'])

print(encoder.inverse_transform([0,1,2,3,4,5]))

df_clone_3['감정_대분류'] = labels

print(df_clone_3['감정_대분류'].value_counts())

['기쁨' '당황' '분노' '불안' '상처' '슬픔']
4    13505
5    12708
2    12503
0    12449
3    12383
1    11308
Name: 감정_대분류, dtype: int64
['기쁨' '당황' '분노' '불안' '상처' '슬픔']
3    7328
2    6920
5    6915
0    6730
4    6624
1    6360
Name: 감정_대분류, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
import re

def apply_regular(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
    result = hangul.sub('', text)
    return result

In [ ]:
df_clone_1['sentence_2'] = df_clone_1['sentence'].apply(apply_regular)

df_clone_1

df_clone_3['sentence_2'] = df_clone_3['sentence'].apply(apply_regular)

df_clone_3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,감정_대분류,sentence,sentence_2
0,0,아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점...,아내가 드디어 출산하게 되어서 정말 신이 나 아 지금 정말 신이 나 아기가 점점 클...
1,3,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. 건강할 때 ...,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야 건강할 때 관...
2,1,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. 아직 학기 ...,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워 아직 학기 초...
3,0,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. ...,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해 퇴직...
4,0,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 빚도 다 갚았으니 당분간은 아무...,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어 빚도 다 갚았으니 당분간은 아무 ...
...,...,...,...
40874,3,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어 내가 주인인가 혼란스...
40875,0,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어. 결과가 좋게 나와서 다...,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어 결과가 좋게 나와서 다행...
40876,5,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼. 내 재능...,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래 너무 슬퍼 내 재능이 ...
40877,0,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야. 덕분에 건강...,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야 덕분에 건강도...


In [ ]:
df_clone_1.to_csv('/content/drive/MyDrive/Colab Notebooks/P4/train_data.tsv', sep='\t')
df_clone_3.to_csv('/content/drive/MyDrive/Colab Notebooks/P4/test_data.tsv', sep='\t')

In [ ]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Colab Notebooks/P4/test_data.tsv", field_indices=[1, 0], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Colab Notebooks/P4/test_data.tsv", field_indices=[1, 0], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

ValueError: ignored

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))